In [ ]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt

# Load the specified MAT-file using scipy.io.loadmat()
file_path = 'data/pendulum_data/data_pendulum_210.mat'
data_pendulum = loadmat(file_path)

In [ ]:
# Assuming the loaded MAT-file is named data_pendulum_90
X = data_pendulum['DATA_X']
Y = data_pendulum['DATA_Y']

print(X.shape)


In [ ]:
len_all = X.shape[0]
data_x_train = X[:int(0.7*len_all)]
data_x_valid = X[int(0.7*len_all)+1:]

data_y_train = Y[:int(0.7*len_all)]
data_y_valid = Y[int(0.7*len_all)+1:]

data_train = [data_x_train, data_y_train]
data_valid = [data_x_valid, data_y_valid]

Train the basis using EDMD-DL and NN-resDMD.

In [ ]:
### Calculate for chosen number of basis ###
# Results from solver_edmd/solver_resdmd
from solver_resdmd import KoopmanNN, KoopmanSolver
# from solver_edmd import KoopmanNN, KoopmanSolver

basis_function = KoopmanNN(layer_sizes=[100,100,100], n_psi_train=47) # basis number would be 100

solver = KoopmanSolver(dic=basis_function,
                         target_dim=np.shape(data_x_train)[-1],
                         reg=0.1)
solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=200,
             batch_size=100000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

# Results from solver_edmd/solver_resdmd
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]
Psi_X = solver.get_Psi_X().numpy()
Psi_Y = solver.get_Psi_Y().numpy()
Koopman_matrix_K = solver.K.numpy()
# Psi_X2, Psi_Y2 = solver.get_basis(data_x_train, data_y_train)
# Psi_X2 = Psi_X2.numpy()
# Psi_Y2 = Psi_Y2.numpy()
# jacobian, hessian = solver.get_derivatives(data_x_valid) # Currently the storage is not enough to calculate for larger amount of data
# np_config.enable_numpy_behavior()
outputs = {
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict': N_dict,
    'K': Koopman_matrix_K,
    'Psi_X': Psi_X,
    'Psi_Y': Psi_Y,
    # 'Psi_X2': Psi_X2,
    # 'Psi_Y2': Psi_Y2,
    # "Jacobian": jacobian,  # Uncomment if needed
    # "Hessian": hessian,    # Uncomment if needed
}

In [ ]:

# Assuming evalues is a numpy array of complex numbers
real_parts = evalues.real
imag_parts = evalues.imag

# Create the plot
plt.figure(figsize=(8, 6))
plt.scatter(real_parts, imag_parts, color='blue', label='Eigenvalues')

# Draw a unit circle for reference
theta = np.linspace(0, 2*np.pi, 100)
plt.plot(np.cos(theta), np.sin(theta), linestyle='--', color='grey', label='Unit Circle')

plt.title('Eigenvalues in Unit Circle')
plt.xlabel('Real Part')
plt.ylabel('Imaginary Part')
plt.axis('equal')  # Ensure the aspect ratio is equal to make the unit circle round
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend()
plt.show()

In [ ]:
### Calculate for multiple numbers of basis ###
import os
import scipy.io as sio

# from solver_edmd import KoopmanNN, KoopmanSolver
from solver_resdmd import KoopmanNN, KoopmanSolver

n_psi_train = [297, 247, 197, 147, 97, 47] # basis number would be 25, 50, 100, 150, 200, 250, 300

for n_psi in n_psi_train:
    # Initialize the basis function with the current value of n_psi_train
    basis_function = KoopmanNN(layer_sizes=[300, 300, 300], n_psi_train=n_psi)

    # Initialize the solver with the basis function and other parameters
    solver = KoopmanSolver(dic=basis_function,
                             target_dim=np.shape(data_x_train)[-1],
                             reg=0.1)
    # Build the solver with the specified training and validation data
    solver.build(data_train=data_train,
                 data_valid=data_valid,
                 epochs=3,
                 batch_size=50000,
                 lr=1e-4,
                 log_interval=10,
                 lr_decay_factor=.8)

    # Assuming the user has these functions or similar ones defined in their environment
    evalues = solver.eigenvalues.T
    efuns = solver.eigenfunctions(X)
    kpm_modes = solver.compute_mode().T
    N_dict = np.shape(evalues)[0]
    Koopman_matrix_K = solver.K.numpy()    
    Psi_X = solver.get_Psi_X().numpy()
    Psi_Y = solver.get_Psi_Y().numpy()   
    # Psi_X2, Psi_Y2 = solver.get_basis(data_x_train, data_y_train) # basis are orthonormal
    # Psi_X2 = Psi_X2.numpy()
    # Psi_Y2 = Psi_Y2.numpy() 
    # jacobian, hessian = solver.get_derivatives(data_x_valid) # Currently the storage is not enough to calculate for larger amount of data
    # np_config.enable_numpy_behavior()
    outputs = {
        'efuns': efuns,
        'evalues': evalues,
        'kpm_modes': kpm_modes,
        'N_dict': N_dict,
        'K': Koopman_matrix_K,
        'Psi_X': Psi_X,
        'Psi_Y': Psi_Y,
        # 'Psi_X2': Psi_X2,
        # 'Psi_Y2': Psi_Y2,
        # "Jacobian": jacobian,  # Uncomment if needed
        # "Hessian": hessian,    # Uncomment if needed
    }

    folder_path = 'data/pendulum_data'  # Adjust the path as needed if you want a different location
    os.makedirs(folder_path, exist_ok=True)
    # Save the outputs to a .mat file named according to the number of basis functions, inside the "data" folder
    file_path = os.path.join(folder_path, f'pendulum_210_resdmd_{N_dict}basis.mat')
    sio.savemat(file_path, outputs)
    print(f'Saved outputs for n_psi_train={n_psi} to {file_path}')